In [ ]:
import cv2
import pickle
import os.path
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,MaxPooling2D,Dense,Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import model_from_json
from sklearn import metrics
from sklearn.metrics import roc_curve,auc

In [ ]:
json_file = open("model1.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
# load weights into new model
loaded_model.load_weights("model1.h5")
loaded_model.compile(loss='sparse_categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
print("Loaded model from disk")

In [ ]:
from helpers import resize_to_fit
path='/Validation'
data1=[]
labels1=[]
for image_file in paths.list_images(path):
    image=cv2.imread(image_file)
    image=resize_to_fit(image,128,128)
    label=image_file.split(os.path.sep)[-2]
    data1.append(image)
    labels1.append(label)
data1 = np.array(data1, dtype="float") / 255.0
labels1 = np.array(np.expand_dims(labels1, axis=1))

print(data1.shape)
print(labels1.shape)

In [ ]:
lb=LabelBinarizer().fit(labels1)
labels1=lb.transform(labels1)
print(labels1.shape)
print(labels1)

In [ ]:
scores =loaded_model.evaluate(data1,labels1, verbose=2)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
y_preds=loaded_model.predict(data1)
y_classes=y_preds.argmax(axis=-1)

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve(labels1,y_preds[:,1], pos_label=1)
roc_auc = auc(fpr, tpr)
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b',label='AUC = %0.2f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.1,1.2])
plt.ylim([-0.1,1.2])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(labels1,y_classes))